In [1]:
## Read SEC data and append data from APIs 

In [2]:
import pandas as pd

from pathlib import Path
import csv

import os
import requests
import json

import quandl

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## Constants 

## Set start date variable - dataframes will be created starting from this date
start_date = '2016-01-01'
end_date = '2021-10-29'
#default_date_range = '2y' ## Default Range for IEX functions - don't need more at the moment

In [4]:
## QUANDL/NASDAQ 
nsdq_api_key = os.environ.get('NASDAQ_API_KEY')
base_url_nsdq = 'https://data.nasdaq.com/api/v3/datasets/FINRA/'

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [5]:
## Get Short "Interest" Data from Quandl 
def get_short_data_QUANDL(symbol):
    string_nsdq = "FINRA/FNSQ_"+symbol
    string_nyse = "FINRA/FNYX_"+symbol
    
    df1 = quandl.get(string_nsdq,start_date=start_date,end_date=end_date,authtoken=nsdq_api_key)   ## Nasdaq
    df2 = quandl.get(string_nyse,start_date=start_date,end_date=end_date,authtoken=nsdq_api_key)   ## NYSE

    df1 = df1.rename(columns={'ShortVolume':'ShortVolumeNSDQ','TotalVolume':'TotalVolumeNSDQ'})
    #df1 = df1.drop(columns={'ShortExemptVolume'})
    df1 = df1.rename(columns={'ShortExemptVolume':'ShortExemptVolumeNSDQ'})

    df2 = df2.rename(columns={'ShortVolume':'ShortVolumeNYSE','TotalVolume':'TotalVolumeNYSE'})
    #df2 = df2.drop(columns={'ShortExemptVolume'})
    df2 = df2.rename(columns={'ShortExemptVolume':'ShortExemptVolumeNYSE'})

    df3 = pd.merge(df1,df2,on='Date',how='outer')
    
    return df3


## Return FTD Data from SEC FTD files using a Stock's CUSIP number to sort 
def return_ftd_data_cusip(cusip_number):
    df = ftd_df.copy()
    df.set_index("CUSIP",inplace=True)
    df = df.loc[cusip_number]
    df = df.set_index('Date')
    return df


## Return the CUSIP symbol from the symbol_df symbol list 
def return_CUSIP_from_symbol(symbol):
    df = symbol_df.copy()
    df.set_index('SYMBOL',inplace=True)
    cusip_variable = df.loc[symbol]
    cusip_variable = cusip_variable['CUSIP']
    return cusip_variable

def return_ftd_data_symbol(symbol):
    cusip_number = return_CUSIP_from_symbol(symbol)
    df = return_ftd_data_cusip(cusip_number)
    return df

In [6]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

## FMP Functions 
def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
    url_hist_price = fmpbase_urlv3+'historical-price-full/'
    url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
    resp_data = requests.get(url_hist_query_with_date)
    json_ = resp_data.json()
    data = json_['historical']
    df = pd.DataFrame(data)
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
    df.set_index('Date',inplace=True)
    df.drop(columns='label',inplace=True)
    return df

api_key = os.getenv("FMP_CLOUD_API_KEY")
def get_float_data_FMP(symbol):
    url_float_shares = fmpbase_urlv4+'shares_float?symbol='
    url_query_float_data = url_float_shares+symbol+'&apikey='+api_key
    resp_data = requests.get(url_query_float_data)
    #df = pd.DataFrame(resp_data.json())
    json_ = resp_data.json()
    return json_[0]

def get_company_profile_FMP_json(symbol):
    ## https://fmpcloud.io/api/v3/profile/AAPL?apikey='yourkeyhere'
    url_company_profile_url = fmpbase_urlv3+'profile/'+symbol+'?apikey='+api_key
    resp_data = requests.get(url_company_profile_url)
    json_response = resp_data.json()
    return json_response[0]

def save_and_export_raw_df_csv(data, symbol):
    path = ('../FilesExportIndividualStockDFs/'+symbol+'_combined_df.csv')
    data.to_csv(path)

In [7]:
## Load IEX to get ETF statistics 

token_status = test_token ## Set to either real token or test token for IEX

def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [8]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [9]:
## Import FTD File using CSV 
ftd_df = pd.read_csv(
    Path('../Resources/ftd_all_data.csv'),
    index_col=0, parse_dates=True
)

In [10]:
## Import Symbol and CUSIP list using CSV
symbol_df = pd.read_csv(
    Path('../Resources/symbol_all_list.csv'),
    index_col=0
)

In [11]:
ftd_df

,Date,CUSIP,SYMBOL,QUANTITY_FAILS
0,2017-09-11,Y21990034,0034RIGHTS,103745.0
1,2017-09-12,Y21990034,0034RIGHTS,103745.0
2,2018-10-17,812350122,0122PIK,828.0
3,2018-10-18,812350122,0122PIK,828.0
4,2019-05-02,812350122,0122PIK,828.0
...,...,...,...,...
7022993,2021-05-25,98959W203,ZZZOD,324.0
7022994,2021-05-26,98959W203,ZZZOD,354.0
7022995,2021-05-27,98959W203,ZZZOD,976.0
7022996,2021-05-28,98959W203,ZZZOD,2.0


In [12]:
symbol_df

,SYMBOL,CUSIP
0,0034RIGHTS,Y21990034
1,0122PIK,812350122
2,0297RTS,G72990297
3,0329REORG,G33990329
4,0888RTSPYMNT,529900888
...,...,...
33911,ZZLL,98880P103
33912,ZZLL,98880P202
33913,ZZLLD,98880P202
33914,ZZZOD,98959W203


In [15]:
test_symbol = 'GME'
symbol = test_symbol
symbol

'GME'

In [18]:
ftd_data = return_ftd_data_symbol(symbol)
ftd_data = ftd_data.drop(columns={'SYMBOL'})
ftd_data

,QUANTITY_FAILS
Date,
2016-01-04,9.0
2016-01-05,2709.0
2016-01-06,129.0
2016-01-07,1733.0
2016-01-08,187.0
...,...
2021-10-22,5680.0
2021-10-25,11251.0
2021-10-27,67280.0


In [19]:
fmp_data = get_FMP_historical_data(symbol)
fmp_data

TypeError: can only concatenate str (not "NoneType") to str

In [20]:
# def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
#     url_hist_price = fmpbase_urlv3+'historical-price-full/'
#     url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
#     resp_data = requests.get(url_hist_query_with_date)
#     json_ = resp_data.json()
#     data = json_['historical']
#     df = pd.DataFrame(data)
#     df.rename(columns={'date':'Date'},inplace=True)
#     df['Date'] = pd.to_datetime(df['Date'])
#     df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
#     df.set_index('Date',inplace=True)
#     df.drop(columns='label',inplace=True)
#     return df

In [ ]:
# url_hist_price = fmpbase_urlv3+'historical-price-full/'
# url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
# resp_data = requests.get(url_hist_query_with_date)
# json_ = resp_data.json()
# data = json_['historical']
# df = pd.DataFrame(data)
# df.rename(columns={'date':'Date'},inplace=True)
# df['Date'] = pd.to_datetime(df['Date'])
# df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
# df.set_index('Date',inplace=True)
# df.drop(columns='label',inplace=True)